# Get Photometry Tables from CDS and adding the Bibcodes for each filter

From the objects that we find in NED with Ne emission lines in MIR, now we are going to get their corresponding phot tables from [CDS (Vizier)](http://vizier.unistra.fr/vizier/sed/?-c=&-c.r=10&-c.u=arcsec). The code to download and check the VOTables from CDS is based in Andres Ramos's work (https://github.com/aframosp/AGNView). Also the part to add the bibcodes to the CDS photometry tables

To reach this aim, in the function **photCDS** using with the tools **request.get** of **[Python Request](https://docs.python-requests.org/en/latest/)** we search the photometry through the vizier url address defined before for each object of the SIMBAD sample (first notebook) and save it in a vot file. Then with the function **check_vot** we check that the content of the votable is OK. If not, the function using tools like **Session** and **Request** deletes the content-length to try to get again the votable.


In [1]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [2]:
from datetime import date
import time
import requests
import numpy as np

from astroquery.vizier import Vizier

from pathlib import Path
from requests import Request, Session

import astropy.units as u
from astropy.table import hstack, QTable, Table

In the next cell we define the paths and create the folders where we will save the data downloaded from the CDS:

In [3]:
path_raw='../Datos/Raw/'
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/' #In this path wi will save the photometry tables for each object
Path(path_phot_cds).mkdir(parents=True, exist_ok=True)
path_cds_vot = '../Datos/Work/Phot_Tables_CDS/VOT/' 
Path(path_cds_vot).mkdir(parents=True, exist_ok=True)

In [4]:
def photCDS(url,name,path):
    req = requests.get(url,
                       headers={'cache-control': 'private, max-age=0, no-cache', "Pragma": "no-cache"})
    if req.status_code == 200:
        with open(path+name+'_CDS_votables.vot', 'wb') as f_vot:
            f_vot.write(req.content)
            f_vot.close()
    check_vot(url,name,path)
    return req

def check_vot(url,name,path):
    """Check that the content of the votable is ok"""
    vot_bad = True
    while vot_bad:
        try:
            cds_VOTable = Table.read(path+name+'_CDS_votables'+'.vot',
                                        format='votable')
            print('Votable checked')
            vot_bad = False
        except ValueError:
            print('Error in votable, deleting content-length')
            ses = Session()
            req_p = Request('POST', url)
            prepped = req_p.prepare()
            del prepped.headers['content-length']
            ses.send(prepped)
            photCDS(url,name,path)

def add_bibcode_cds(name,path):
    '''
    Adding the Bibcode to the CDS tables
    '''
    cds_votable = Table.read(path+name+'_CDS_votables'+'.vot',
                             format='votable')                           
    cds_votable['Bibcode'] = np.array(['Empty']*len(cds_votable),
                                         dtype='object')
    print('Searching for Bibcode')
    for tabindx, tabinfo in enumerate(cds_votable['_tabname']):
        try:
            time.sleep(0.001)  # This is used to avoid being flag by the server
            name_cat = tabinfo.rpartition('/')[0]
            search = Vizier.query_constraints(catalog='METAcat', name=name_cat)
            cds_votable['Bibcode'][tabindx] = search[0][0]['bibcode']
        except IndexError:
            print('There is an error at ', tabindx, tabinfo)
    print('Adding Bibcode to ',name, ' votable')        
    return cds_votable

In [5]:
date_sample = 'Apr_12_2022'

In [7]:
NeIR_obj_t = Table.read(path_raw+'Obj_Ne-IR_'+date_sample+'.vot',format='votable')
NeIR_obj_t.show_in_notebook()

idx,Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index,Ne_IR_info,NED_Ne_IR_info
0,3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0,NeII|NeIII|NeV,NeII|NeIII|NeV
1,M 84,M 84,186.2655972083333,12.886983138888887,Sy2,0.00327,z,* |AGN|G |GiC|GiG|IR |Rad|Sy2|UV |X |smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=186.2655972083333%2012.886983138888887&-c.rs=5,414.0,NeV,NeV
2,2MASS J13000535+1632148,NVSS J130005+163212,195.02228295163,16.53743305218,Sy1,0.07993,z,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5,428.0,NeII|NeIII,NeII|NeIII
3,NGC 262,NGC 262,12.196426138730002,31.956956641230004,Sy2,0.015260233939595302,v,* |AGN|Bla|G |GiG|IR |QSO|Rad|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=12.196426138730002%2031.956956641230004&-c.rs=5,484.0,NeII|NeIII|NeV,NeII|NeIII|NeV
4,IRAS 04385-0828,IRAS 04385-0828,70.2290037068,-8.37281297812,Sy2,0.015064,z,* |AGN|G |GiG|IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=70.2290037068%20-8.37281297812&-c.rs=5,541.0,NeII|NeIII|NeV,NeII|NeIII|NeV
5,NGC 4945,NGC 4945,196.3636625,-49.4679,Sy2,0.0017968563040435814,v,AGN|G |GiG|IR |Rad|Sy2|X |cm |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=196.3636625%20-49.4679&-c.rs=5,628.0,NeII|NeIII|NeV,NeII|NeIII|NeV
6,IC 4553,IC 4553,233.73843316666665,23.50322455555555,SyG,0.0184,z,AGN|G |GiG|GiP|IG |IR |LIN|Rad|SyG|X |gam|smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=233.73843316666665%2023.50322455555555&-c.rs=5,654.0,NeII|NeIII|NeV,NeII|NeIII|NeV
7,NGC 5252,NGC 5252,204.56612821846997,4.542644796,Sy2,0.02294,z,AG?|AGN|EmG|G |GiG|Sy1|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=204.56612821846997%204.542644796&-c.rs=5,773.0,NeII,NeII
8,Mrk 279,Mrk 279,208.26431206837998,69.30816969735,Sy1,0.030601,z,AGN|G |IR |QSO|Sy1|UV |X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=208.26431206837998%2069.30816969735&-c.rs=5,852.0,NeII|NeIII|NeV,NeII|NeIII|NeV
9,UGC 11680,UGC 11680,316.93375,3.875277777777778,Sy2,0.026,z,G |GiP|IG |IR |PaG|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=316.93375%203.875277777777778&-c.rs=5,1227.0,NeII,NeII


### Downloading the CDS Tables:

In [9]:
start_cds = time.time()
for i, id_ in enumerate(NeIR_obj_t['Main_Id']):
    print(i,' ',id_,' getting CDS Phot Table')
    cds_req = photCDS(NeIR_obj_t['cds_url'][i],id_,path_cds_vot)
    print(id_,' CDS VOTable saved OK')
    cds_table = add_bibcode_cds(id_,path_cds_vot)
    cds_table.write(path_phot_cds+id_+'_CDS_phot_tables'+'.vot', #Save the phot table in a vot table file
                         format='votable',overwrite=True)
    print(id_,' CDS Phot Table saved OK\n')
end_cds = time.time()

print('\nExecution Time(seg): ' + str(end_cds - start_cds))
print('Execution Time(min): ' + str((end_cds - start_cds)/60))
print('Execution Time(hrs): ' + str((end_cds - start_cds)/3600))

0   3C 234.0  getting CDS Phot Table
Votable checked
3C 234.0  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  3C 234.0  votable
3C 234.0  CDS Phot Table saved OK

1   M  84  getting CDS Phot Table
Votable checked
M  84  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  M  84  votable
M  84  CDS Phot Table saved OK

2   2MASS J13000535+1632148  getting CDS Phot Table
Votable checked
2MASS J13000535+1632148  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASS J13000535+1632148  votable
2MASS J13000535+1632148  CDS Phot Table saved OK

3   NGC   262  getting CDS Phot Table
Votable checked
NGC   262  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC   262  votable
NGC   262  CDS Phot Table saved OK

4   IRAS 04385-0828  getting CDS Phot Table
Votable checked
IRAS 04385-0828  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IRAS 04385-0828  votable
IRAS 04385-0828  CDS Phot Table saved OK

5   NGC  4945  getting CDS Pho

Votable checked
IC 3639  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IC 3639  votable
IC 3639  CDS Phot Table saved OK

43   Mrk  926  getting CDS Phot Table
Votable checked
Mrk  926  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  926  votable
Mrk  926  CDS Phot Table saved OK

44   Mrk  463E  getting CDS Phot Table
Votable checked
Mrk  463E  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  463E  votable
Mrk  463E  CDS Phot Table saved OK

45   NGC  4235  getting CDS Phot Table
Votable checked
NGC  4235  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  4235  votable
NGC  4235  CDS Phot Table saved OK

46   IRAS 15480-0344  getting CDS Phot Table
Votable checked
IRAS 15480-0344  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IRAS 15480-0344  votable
IRAS 15480-0344  CDS Phot Table saved OK

47   UGC  6728  getting CDS Phot Table
Votable checked
UGC  6728  CDS VOTable saved OK
Searching for Bibcode
A

Votable checked
NGC  2639  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  2639  votable
NGC  2639  CDS Phot Table saved OK

86   NGC   513  getting CDS Phot Table
Votable checked
NGC   513  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC   513  votable
NGC   513  CDS Phot Table saved OK

87   NAME IRAS F15163+4255 NW  getting CDS Phot Table
Votable checked
NAME IRAS F15163+4255 NW  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NAME IRAS F15163+4255 NW  votable
NAME IRAS F15163+4255 NW  CDS Phot Table saved OK

88   ESO 383-35  getting CDS Phot Table
Votable checked
ESO 383-35  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  ESO 383-35  votable
ESO 383-35  CDS Phot Table saved OK

89   NAME IRAS 09111-1007W  getting CDS Phot Table
Votable checked
NAME IRAS 09111-1007W  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NAME IRAS 09111-1007W  votable
NAME IRAS 09111-1007W  CDS Phot Table saved OK

90   NGC   9

Votable checked
NGC  4138  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  4138  votable
NGC  4138  CDS Phot Table saved OK

127   NGC  1566  getting CDS Phot Table
Votable checked
NGC  1566  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  1566  votable
NGC  1566  CDS Phot Table saved OK

128   NGC  1808  getting CDS Phot Table
Votable checked
NGC  1808  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  1808  votable
NGC  1808  CDS Phot Table saved OK

129   ICRF J054236.1+495107  getting CDS Phot Table
Votable checked
ICRF J054236.1+495107  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  ICRF J054236.1+495107  votable
ICRF J054236.1+495107  CDS Phot Table saved OK

130   NGC  4074  getting CDS Phot Table
Votable checked
NGC  4074  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  4074  votable
NGC  4074  CDS Phot Table saved OK

131   Mrk  304  getting CDS Phot Table
Votable checked
Mrk  304  CDS VOT

Votable checked
ESO 420-13  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  ESO 420-13  votable
ESO 420-13  CDS Phot Table saved OK

169   UGC  1395  getting CDS Phot Table
Votable checked
UGC  1395  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  UGC  1395  votable
UGC  1395  CDS Phot Table saved OK

170   IC 4518A  getting CDS Phot Table
Votable checked
IC 4518A  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IC 4518A  votable
IC 4518A  CDS Phot Table saved OK

171   NAME Centaurus A  getting CDS Phot Table
Votable checked
NAME Centaurus A  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NAME Centaurus A  votable
NAME Centaurus A  CDS Phot Table saved OK

172   2XMM J094506.4+035552  getting CDS Phot Table
Votable checked
2XMM J094506.4+035552  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2XMM J094506.4+035552  votable
2XMM J094506.4+035552  CDS Phot Table saved OK

173   ICRF J052109.8+163822  getting CDS Phot

Votable checked
NGC  3976  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  3976  votable
NGC  3976  CDS Phot Table saved OK

211   Ton  951  getting CDS Phot Table
Votable checked
Ton  951  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Ton  951  votable
Ton  951  CDS Phot Table saved OK

212   M  58  getting CDS Phot Table
Votable checked
M  58  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  M  58  votable
M  58  CDS Phot Table saved OK

213   2dFGRS TGN336Z243  getting CDS Phot Table
Votable checked
2dFGRS TGN336Z243  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2dFGRS TGN336Z243  votable
2dFGRS TGN336Z243  CDS Phot Table saved OK

214   LEDA 1487448  getting CDS Phot Table
Votable checked
LEDA 1487448  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA 1487448  votable
LEDA 1487448  CDS Phot Table saved OK

215   NGC  4395  getting CDS Phot Table
Votable checked
NGC  4395  CDS VOTable saved OK
Searchin

Votable checked
Mrk   79  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk   79  votable
Mrk   79  CDS Phot Table saved OK

254   LEDA   68751  getting CDS Phot Table
Votable checked
LEDA   68751  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA   68751  votable
LEDA   68751  CDS Phot Table saved OK

255   NGC  7319  getting CDS Phot Table
Votable checked
NGC  7319  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  7319  votable
NGC  7319  CDS Phot Table saved OK

256   Mrk  863  getting CDS Phot Table
Votable checked
Mrk  863  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  863  votable
Mrk  863  CDS Phot Table saved OK

257   2MASX J18334627+4727027  getting CDS Phot Table
Votable checked
2MASX J18334627+4727027  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  2MASX J18334627+4727027  votable
2MASX J18334627+4727027  CDS Phot Table saved OK

258   Mrk 1066  getting CDS Phot Table
Votable checked
Mrk 

Votable checked
UGC  3351  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  UGC  3351  votable
UGC  3351  CDS Phot Table saved OK

296   Mrk  705  getting CDS Phot Table
Votable checked
Mrk  705  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk  705  votable
Mrk  705  CDS Phot Table saved OK

297   UGC  3601  getting CDS Phot Table
Votable checked
UGC  3601  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  UGC  3601  votable
UGC  3601  CDS Phot Table saved OK

298   3C 286  getting CDS Phot Table
Votable checked
3C 286  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  3C 286  votable
3C 286  CDS Phot Table saved OK

299   ESO  33-2  getting CDS Phot Table
Votable checked
ESO  33-2  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  ESO  33-2  votable
ESO  33-2  CDS Phot Table saved OK

300   NGC  4725  getting CDS Phot Table
Votable checked
NGC  4725  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  47

Votable checked
LEDA 2817285  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LEDA 2817285  votable
LEDA 2817285  CDS Phot Table saved OK

338   NGC  7213  getting CDS Phot Table
Votable checked
NGC  7213  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  NGC  7213  votable
NGC  7213  CDS Phot Table saved OK

339   Mrk 1506  getting CDS Phot Table
Votable checked
Mrk 1506  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  Mrk 1506  votable
Mrk 1506  CDS Phot Table saved OK

340   IRAS F00183-7111  getting CDS Phot Table
Votable checked
IRAS F00183-7111  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  IRAS F00183-7111  votable
IRAS F00183-7111  CDS Phot Table saved OK

341   LBQS 1244+0238  getting CDS Phot Table
Votable checked
LBQS 1244+0238  CDS VOTable saved OK
Searching for Bibcode
Adding Bibcode to  LBQS 1244+0238  votable
LBQS 1244+0238  CDS Phot Table saved OK

342   NGC  6860  getting CDS Phot Table
Votable checked
NGC  6860  

### Notebook Info:

In [28]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 8.2.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.13.0-39-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.0

sys     : 3.9.7 (default, Sep 16 2021, 13:09:58) 
[GCC 7.5.0]
astropy : 5.0
requests: 2.27.1
numpy   : 1.20.3

Watermark: 2.3.0

